# Sin Filtro (TP2 Orga2)
## Experimentos de Bordes 💪💪

### Compilado de código

In [ ]:
!cp -a ../src/img img
!cd ../src && make all
!cp ../src/build/tp2 .
!cp ../src/build/tp2_im2 .
!cp ../src/build/tp2_im4 .
!cp ../src/build/tp2_im8 .

### Imports

In [ ]:
# Verifico la correcta instalación. Si no falla el import está OK
#!/usr/bin/env python3
!pwd
!python3 --version

#import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from IPython.display import Markdown, display
from IPython.display import clear_output

from subprocess import check_output

from ipywidgets import IntProgress, HTML, VBox

### Utilidades

In [ ]:
def logProgress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )
        
def done():
    print("Done!\n\n✨\n  ⊂_ヽ\n 　 ＼＼ \n　　 ＼( ͡° ͜ʖ ͡°)\n　　　 >　⌒ヽ\n　　　/ 　 へ＼\n　　 /　　/　＼＼\n　　 ﾚ　ノ　　 ヽ_つ\n　　/　/\n　 /　/|\n　(　(ヽ\n　|　|、＼\n　| 丿 ＼ ⌒)\n　| | 　　) /\nノ )　　Lﾉ\n(_／)")
    
def rmse(errorCuadratico, n):
    return np.sqrt(1/n * errorCuadratico)

In [ ]:
def box_plot(impls, title):
    ## Create boxplots
    # Create a figure instance
    fig = plt.figure(1, figsize=(9, 6))

    # Create an axes instance
    ax = fig.add_subplot(111)

    dataToPlot = [imp["data"] for imp in impls]

    # Create the boxplot
    ## add patch_artist=True option to ax.boxplot() 
    ## to get fill color
    bp = ax.boxplot(dataToPlot, patch_artist=True, showfliers=False)

    ## change outline color, fill color and linewidth of the boxes
    for box in bp['boxes']:
        # change outline color
        box.set( color='#7570b3', linewidth=2)
        # change fill color
        box.set( facecolor = '#1b9e77' )

    ## change color and linewidth of the whiskers
    for whisker in bp['whiskers']:
        whisker.set(color='#7570b3', linewidth=2)

    ## change color and linewidth of the caps
    for cap in bp['caps']:
        cap.set(color='#7570b3', linewidth=2)

    ## change color and linewidth of the medians
    for median in bp['medians']:
        median.set(color='#b2df8a', linewidth=2)

    ## change the style of fliers and their fill
    for flier in bp['fliers']:
        flier.set(marker='o', color='#e7298a', alpha=0.5)

    ax.set_xticklabels([imp["label"] for imp in impls])

    ## Remove top axes and right axes ticks
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

    ax.set_title(title)

### Obtención de Datos

In [ ]:
IMPS = {
    "asm":      {"bin": "tp2",        "label": "asm",       "data": []},
    "asm_2x":   {"bin": "tp2_im2",    "label": "asm 2x",       "data": []},
    "asm_4x":   {"bin": "tp2_im4",    "label": "asm 4x",       "data": []},
    "asm_8x":   {"bin": "tp2_im8",    "label": "asm 8x",       "data": []}
}

FILTER = "Bordes"
ITER   = "1000"
IMG    = "img/puente.bmp"

def run(im, img:str, t:str=ITER):
    """Corre una implementacion y retorna el output."""
    # Quiero correr
    #  ./tp2 Rombos -i asm -t 100 img/puente.bmp
    return check_output(["./"+im["bin"], FILTER, "-i", "asm", "-t", t, img]).decode("utf-8")

def parse_cycles(out):
    """ Output esperado:
0    Procesando...
1      Filtro             : Rombos
2      Implementación     : C
3      Archivo de entrada : img/puente.bmp
4    Tiempo de ejecución:
5      Comienzo                          : 418148872718872
6      Fin                               : 418154435946972
7      # iteraciones                     : 100
8      # de ciclos insumidos totales     : 5563228100
9      # de ciclos insumidos por llamada : 55632280.000
    """
    return float(out.split('\n')[9].split(': ')[1])

In [ ]:
for im in logProgress(IMPS.keys(), name="Implementación"):
    #for _ in logProgress(range(0,100), name="Iteración"):
        out = run(IMPS[im], img="img/puente.bmp", t="20")
        cycles = parse_cycles(out)
        IMPS[im]["data"].append(cycles)

### Gráficos

In [ ]:
box_plot(IMPS.values(), "Ciclos por implementación y optimización")

### Borrado de binarios

In [1]:
!rm -r img/
!rm *.bmp
!rm -rf tp2*